# Semantička jezgra

U ovom primjeru koda koristit ćete [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI okvir za stvaranje osnovnog agenta.

Cilj ovog primjera je pokazati korake koje ćemo kasnije koristiti u dodatnim primjerima koda prilikom implementacije različitih agentnih obrazaca.


## Uvoz potrebnih Python paketa


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Kreiranje klijenta

U ovom primjeru koristit ćemo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) za pristup LLM-u.

`ai_model_id` je definiran kao `gpt-4o-mini`. Pokušajte promijeniti model na neki drugi model dostupan na tržištu GitHub Models kako biste vidjeli različite rezultate.

Kako bismo koristili `Azure Inference SDK`, koji se koristi za `base_url` za GitHub Models, upotrijebit ćemo konektor `OpenAIChatCompletion` unutar Semantic Kernel-a. Također postoje i drugi [dostupni konektori](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) za korištenje Semantic Kernel-a s drugim pružateljima modela.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Kreiranje agenta

Ovdje kreiramo agenta pod nazivom `TravelAgent`.

Za ovaj primjer koristimo vrlo jednostavne upute. Možete promijeniti ove upute kako biste vidjeli kako agent reagira drugačije.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Pokretanje Agenta

Sada možemo pokrenuti Agenta definirajući nit tipa `ChatHistoryAgentThread`. Sve potrebne sistemske poruke pružaju se agentu putem argumenta ključne riječi `messages` u invoke_stream.

Nakon što su ove poruke definirane, kreiramo `user_inputs`, što predstavlja ono što korisnik šalje agentu. U ovom slučaju, postavili smo ovu poruku na `Planiraj mi sunčani odmor`.

Slobodno promijenite ovu poruku kako biste vidjeli kako agent različito reagira.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za kritične informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
